Давайте с вами решим многоклассовую задачу, но теперь с использованием более верхнеуровневого фреймворка PyTorch.

Для этого мы возьмём датасет MNIST. Он очень похож на Digits из предыдущего задания и тоже содержит черно-белые рукописные цифры от 0 до 9, только размер уже 28х28 пикселей.

Вообще, MNIST — знаковый датасет для глубокого обучения, именно для него Yahn LeCun разработал первую свёрточную нейросеть LeNet и применил для её обучения механизм обратного распространения ошибки, что дало сильный толчок в развитии нейронных сетей и глубокого обучения в целом.

В этом задании вам придется много опираться на ноутбук ipynb, в котором мы знакомились с основными элементами PyTorch

# Финальное задание PyTorch

Начнем с основных импортов и определения констант - доступного девайса для вычислений и параметров, которые зависят от выборки

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# выбор девайса
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# параметры, которые зависят от нашего датасета
input_size = 784
num_classes = 10

Предлагаем вам попробовать выбрать самим гиперпараметры обучения

In [ ]:
# выберите сами данные параметры
num_epochs = # место для вашего кода
batch_size = # место для вашего кода
learning_rate = # место для вашего кода

Загружаем датасет MNIST - он есть в стандартном наборе датасетов pytorch, выбираем transform=ToTensor(), это важно!

In [ ]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# transform.ToTensor() записывает данные в torch.tensor и нормализовывает данные,
# разделяя значения каждого пикселя на 255

Настраиваем dataloader - объект, который датасет 

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        # сконструируйте свою сеть, опишите слои, которые будут использоваться
        # достаточно будет двух линейных слоев с ReLU после первого,
        # но мы оставляем этот выбор вам, можно составить любую сеть
        # SoftMax указывать не нужно 
    
    def forward(self, x):
        # опишите функцию forward, то есть проход по вашей сети
        # результат применения x к первому слою отдайте на вход функции активации
        # выход функции активации отправьте на вход следущему слою и так далее
        return out

model = NeuralNet(input_size, num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
# в качестве функции потерь используем Cross Entropy
# в PyTorch кросс энтропия реализована с помощью LogSoftmax и 
# Negative Log Likelyhood loss, что на самом деле даёт такой же результат
# Так как мы не указали внутри сети SoftMax и он учтен только в Loss функции,
# при тестировании сети нам нужно учесть этот факт и руками интерпретировать выход сети


optimizer = torch.optim.# выберите оптимизатор (вариантов много - SGD, AdaGrad, Adam, RMSProp и т.д.)
# параметры для оптимизатора, передайте в скобках в предыдущей строке(model.parameters(), lr=learning_rate)  
# почитать про оптимизаторы можно тут https://pytorch.org/docs/stable/optim.html

In [ ]:
# Train
total_step = len(train_loader)
for epoch in range(num_epochs):
    # получаем батч
    for i, (images, labels) in enumerate(train_loader):  
        # запишем данные в строку
        images = images.reshape(# подберите размер ).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprpagation and optimization
        optimizer.zero_grad()
        # считаем градиенты
        loss.backward()
        # обновляем параметры
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:


# test
# так как мы не использовали софтмакс, то берем максимум от выхода сети
# и считаем что индекс этого класса и есть наш ответ
# для интерпретируемости можно при тестировании добавить SM на выход
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(# подберите размер).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
import matplotlib.pyplot as plt
batch_counter = 0

total = 0
correct = 0
for images, labels in test_loader:

    images_flat = images.reshape(-1, 28 * 28).to(device)
    labels = labels.to(device)
    outputs = model(images_flat)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    # отрисуем несколько примеров и выведем наши ответы на них
    if (batch_counter < 3):
        plt.figure(batch_counter)
        plt.imshow(images[0][0])
        print(predicted.data[0])
    batch_counter += 1

print("Test accuracy is {}".format(100 * correct/total))



Если точность на тесте больше 0.8 — поздравляем с выполнением задания!